# SMAI ASSIGNMENT 2 - Problem 4.2
## Srujana Vanka - 2020102005


## To visualize the enclosing boxes.

In [1]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [45]:
def visualize_bboxes_and_thresholds(img, df):
    img_2 = img.copy()
    horizontal_gaps = []  # to store horizontal gap values
    vertical_gaps = []    # to store vertical gap values
    
    for index, rows in df.iterrows():
        top_left = (int(rows['Top-Left'].split(',')[0][1:]), int(rows['Top-Left'].split(',')[1][:-1]))
        bottom_right = (int(rows['Bottom-Right'].split(',')[0][1:]), int(rows['Bottom-Right'].split(',')[1][:-1]))
        cv2.rectangle(img_2, top_left, bottom_right, (255, 0, 0), 2)
        
        if index > 0:
            # Calculate horizontal and vertical gaps relative to the previous box
            prev_rows = df.iloc[index - 1]
            prev_bottom_right = (int(prev_rows['Bottom-Right'].split(',')[0][1:]), int(prev_rows['Bottom-Right'].split(',')[1][:-1]))
            
            horizontal_gap = top_left[0] - prev_bottom_right[0]
            vertical_gap = top_left[1] - prev_bottom_right[1]
            
            horizontal_gaps.append(horizontal_gap)
            vertical_gaps.append(vertical_gap)
    
    # Calculate and print statistics for horizontal and vertical gaps
    horizontal_mean = np.mean(horizontal_gaps)
    vertical_mean = np.mean(vertical_gaps)
    
    print(f"Horizontal Gap Statistics - Mean: {horizontal_mean}, Std Dev: {np.std(horizontal_gaps)}")
    print(f"Vertical Gap Statistics - Mean: {vertical_mean}, Std Dev: {np.std(vertical_gaps)}")

image = cv2.imread('/home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/images/29.jpg')
visualize_bboxes_and_thresholds(image, df)


Horizontal Gap Statistics - Mean: -94.70221327967806, Std Dev: 361.194814496618
Vertical Gap Statistics - Mean: -57.668008048289735, Std Dev: 60.758621637390604


In [40]:

input_dir = '/home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/CSV'
output_dir = '/home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv'

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# List all CSV files in the input directory
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# Iterate through each CSV file
for csv_file in csv_files:
    csv_file_path = os.path.join(input_dir, csv_file)
    df = pd.read_csv(csv_file_path)

    # Approximate thresholds for horizontal and vertical connections
    horizontal_threshold = -50
    vertical_threshold = -55  
    
    # Initialize empty lists for the new columns
    top_list = []
    left_list = []
    bottom_list = []
    right_list = []
    top_box_list = []
    bottom_box_list = []
    left_box_list = []
    right_box_list = []

    # Iterate through the DataFrame to calculate the new columns
    for index, row in df.iterrows():
        # Extract coordinates from Top-Left and Bottom-Right columns
        top_left = eval(row['Top-Left'])
        bottom_right = eval(row['Bottom-Right'])

        # Calculate Top, Left, Bottom, and Right coordinates
        top = top_left[1]
        left = top_left[0]
        bottom = bottom_right[1]
        right = bottom_right[0]

        # Initialize variables to store nearest neighbors
        top_box = None
        bottom_box = None
        left_box = None
        right_box = None

        # Iterate again to find nearest neighbors
        for inner_index, inner_row in df.iterrows():
            if inner_index != index:
                inner_top_left = eval(inner_row['Top-Left'])
                inner_bottom_right = eval(inner_row['Bottom-Right'])

                inner_top = inner_top_left[1]
                inner_left = inner_top_left[0]
                inner_bottom = inner_bottom_right[1]
                inner_right = inner_bottom_right[0]

                # Check if the boxes are horizontally or vertically close
                if abs(inner_top - bottom) <= vertical_threshold:
                    top_box = [np.sqrt((top - inner_bottom) ** 2 + (left - inner_left) ** 2), inner_row['Id']]
                if abs(inner_bottom - top) <= vertical_threshold:
                    bottom_box = [np.sqrt((bottom - inner_top) ** 2 + (right - inner_right) ** 2), inner_row['Id']]
                if abs(inner_left - right) <= horizontal_threshold:
                    right_box = [np.sqrt((top - inner_top) ** 2 + (right - inner_left) ** 2), inner_row['Id']]
                if abs(inner_right - left) <= horizontal_threshold:
                    left_box = [np.sqrt((bottom - inner_bottom) ** 2 + (left - inner_right) ** 2), inner_row['Id']]

        # Append the calculated values to the lists
        top_list.append(top)
        left_list.append(left)
        bottom_list.append(bottom)
        right_list.append(right)
        top_box_list.append(top_box)
        bottom_box_list.append(bottom_box)
        left_box_list.append(left_box)
        right_box_list.append(right_box)

    # Add the new columns to the DataFrame
    df['Top'] = top_list
    df['Left'] = left_list
    df['Bottom'] = bottom_list
    df['Right'] = right_list
    df['Top_box'] = top_box_list
    df['Bottom_box'] = bottom_box_list
    df['Left_box'] = left_box_list
    df['Right_box'] = right_box_list

    # Define the output CSV file path
    output_csv_file = os.path.join(output_dir, f'final_{csv_file}')

    # Save the modified DataFrame as a final CSV file
    df.to_csv(output_csv_file, index=False)
    print(f'Created final CSV: {output_csv_file}')


Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_250.csv
Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_174.csv
Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_29.csv
Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_145.csv
Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_101.csv
Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_68.csv
Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_152.csv
Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_232.csv
Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_201.csv
Created final CSV: /home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv/final_TLBR_159.csv


### Input Data:
We with a bunch of images and CSV files. The CSV files contain information about boxes in those images, like where each box starts and ends.

### Preparation:
Some rules (thresholds) are used to decide when two boxes are close enough to be connected.

### Data Processing (CSV Files):
For each CSV file (which corresponds to an image):
Read the CSV file and get the box information.
Calculate some additional information about each box, like its top, bottom, left, and right edges.

In [43]:
import cv2
import ast

def make_connections(image, df):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_with_boxes = image_rgb.copy()

    for index, row in df.iterrows():
        box_id = int(row['Id'])
        top_left = eval(row['Top-Left'])
        bottom_right = eval(row['Bottom-Right'])
        top = int(row['Top'])
        left = int(row['Left'])
        bottom = int(row['Bottom'])
        right = int(row['Right'])
    
        width = right - left
        height = bottom - top

        top_left = (left, top)
        bottom_right = (right, bottom)

        cv2.rectangle(image_with_boxes, top_left, bottom_right, (255, 0, 0), 2)

        label_position = (left, top - 10)
        cv2.putText(image_with_boxes, str(box_id), label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        top_adjacent_id = int(ast.literal_eval(row['Top_box'])[1]) if not pd.isna(row['Top_box']) else 0
        bottom_adjacent_id = int(ast.literal_eval(row['Bottom_box'])[1]) if not pd.isna(row['Bottom_box']) else 0
        left_adjacent_id = int(ast.literal_eval(row['Left_box'])[1]) if not pd.isna(row['Left_box']) else 0
        right_adjacent_id = int(ast.literal_eval(row['Right_box'])[1]) if not pd.isna(row['Right_box']) else 0

        if top_adjacent_id != 0:
            top_adjacent_row = df[df['Id'] == top_adjacent_id].iloc[0]
            top_adjacent_center = eval(top_adjacent_row['Bottom-Right'])
            cv2.line(image_with_boxes, (left + width // 2, top), top_adjacent_center, (0, 255, 0), 2)

        if bottom_adjacent_id != 0:
            bottom_adjacent_row = df[df['Id'] == bottom_adjacent_id].iloc[0]
            bottom_adjacent_center = eval(bottom_adjacent_row['Top-Left'])
            cv2.line(image_with_boxes, (left + width // 2, bottom), bottom_adjacent_center, (0, 255, 0), 2)

        if left_adjacent_id != 0:
            left_adjacent_row = df[df['Id'] == left_adjacent_id].iloc[0]
            left_adjacent_center = eval(left_adjacent_row['Bottom-Right'])
            cv2.line(image_with_boxes, (left, top + height // 2), left_adjacent_center, (0, 255, 0), 2)

        if right_adjacent_id != 0:
            right_adjacent_row = df[df['Id'] == right_adjacent_id].iloc[0]
            right_adjacent_center = eval(right_adjacent_row['Top-Left'])
            cv2.line(image_with_boxes, (right, top + height // 2), right_adjacent_center, (0, 255, 0), 2)

    return image_with_boxes, df


In [44]:
images_folder = '/home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/images'
final_csv_folder = '/home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/final_csv' 
output_folder = '/home/sruj/Downloads/Dataset_4_2/SMAI - AQ2/output'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all files in the images folder
image_files = os.listdir(images_folder)

for image_file in image_files:
    if image_file.endswith('.jpg'):
        image_path = os.path.join(images_folder, image_file)
        csv_file = 'final_TLBR_' + os.path.splitext(image_file)[0] + '.csv'
        csv_path = os.path.join(final_csv_folder, csv_file)  # Use the final CSV folder

        if os.path.exists(csv_path):
            image = cv2.imread(image_path)
            df = pd.read_csv(csv_path)

            print(f"Processing: {image_file}")

            visualized_image, df_with_connections = make_connections(image, df)

            # Save the visualized image to the output folder
            output_image_path = os.path.join(output_folder, image_file)
            cv2.imwrite(output_image_path, cv2.cvtColor(visualized_image, cv2.COLOR_RGB2BGR))
            output_csv_file = 'Final_' + csv_file  
            output_csv_path = os.path.join(output_folder, output_csv_file)
            df_with_connections.to_csv(output_csv_path, index=False)

Processing: 232.jpg
Processing: 152.jpg
Processing: 145.jpg
Processing: 29.jpg
Processing: 159.jpg
Processing: 101.jpg
Processing: 174.jpg
Processing: 250.jpg
Processing: 201.jpg
Processing: 68.jpg
